# Code Demo Process Mining
Quirin Joshua Groszeibl  
Aktuelle Themen der KI Seminar

In [ ]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

## Daten laden

Zuerst brauchen wir Daten, die wir verwerten können. In diesem Fall laden wir dazu eine CSV Datei hoch. Das Beispiel hierbei ist ein Datensatz von Keggle mit über 27k Reihen Eventlogs einer Autoversicherungsfirma, diese begrenzen wir für diese Demo auf 200 Reihen, damit das Complilieren nicht allzulange dauert.

In [2]:
# Load the event log as a DataFrame
event_log = pd.read_csv('/home/qsh1ne/PM_Code_Demo/Insurance_claims_event_log.csv',
                        nrows=200, usecols=['case_id', 'activity_name', 'timestamp'])

## Daten verstehen

Damit wir eine Fehlerfreie Analyse durchführen können, benötigen wir zuerst ein Verständnis für unsere Daten. Eine Frequenzanalyse hilft uns direkte Zusammenhänge zwischen den Variabeln besser zu verstehen. Für die spätere Visualisierung erstellen wir anschließend einen gerichteten Graph, dessen Knoten wir mit den Subprozessen der Case ID bestücken.

In [ ]:
# Perform frequency analysis to get the directly-follows relations
dfg = event_log.groupby(['case_id', event_log['activity_name'].shift(-1)]).size().reset_index(name='count')

# Create a directed graph
graph = nx.DiGraph()

# Add nodes and edges to the graph
for _, row in dfg.iterrows():
    graph.add_edge(row['activity_name'], row['activity_name'], weight=row['count'])

# Visualize the graph
pos = nx.spring_layout(graph, seed=42)
labels = nx.get_edge_attributes(graph, 'weight')
weights = [graph[u][v]['weight'] / 10 for u, v in graph.edges()]
nx.draw_networkx(graph, pos, with_labels=True, node_size=500, node_color='lightblue',
                 edge_color='gray', width=weights, font_size=8)
nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, font_size=6)


Anschließend lassen wir und den Graphen als Bild ausgeben, um notfalls später auf die Ergebnisse zurückzukommen.

In [ ]:
# Save the process graph as an image file
output_dir = '/home/qsh1ne/PM_Code_Demo/output'
os.makedirs(output_dir, exist_ok=True)
process_graph_file = os.path.join(output_dir, 'process_graph.png')
plt.savefig(process_graph_file)

print(f"Process graph saved as {process_graph_file}")



**Beispiel**: Die EventLogs der Versicherungsfirma beziehen sich auf Cases, welche jeweils in 6 Schritten aufgespalten werden, wie auf der Abbildung unten zu sehen. Anhand der gleichmäßigen Größe aller Subprozesse, lässt sich darauf schließen, dass diese alle in etwa gleich oft vorkommen.

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/Ep2op8c4Gd9Fx6q?x=1920&y=579&a=true&file=process_graph.png&scalingup=0)


## Process Spektrum errechnen

Das Errechnen des Process Spektrums ganz einfach über das Zählen der einzelnen Aktivitäten in der Anzahl. Jeder Subprocess muss einen sehr ähnlichen Ablauf haben, um mit dem PSM gute Vorhersagen treffen zu können. Beispielsweise eignen sich Maschinendaten oder Bürokratische Abläufe sehr gut, während Daten von Sportwettkämpfen hierbei weniger gute Ergebnisse liefern.

Es gilt: **Je ähnlicher die Processe, desto einfach die Auswertung.** 

Auch hier visualisieren wir die Ergebnisse zuerst und speichern diese anschließend als Bild ab.

In [ ]:
# Calculate process spectrum
spectrum = event_log['activity_name'].value_counts().sort_values(ascending=False)

# Plot the process spectrum
plt.figure()
spectrum.plot(kind='bar', color='lightblue')
plt.xlabel('Activity')
plt.ylabel('Frequency')
plt.title('Process Spectrum')

# Save the process spectrum plot as an image file
spectrum_plot_file = os.path.join(output_dir, 'process_spectrum.png')
plt.savefig(spectrum_plot_file)

print(f"Process spectrum saved as {spectrum_plot_file}")



**Beispiel:** Hierbei fällt auch schon der erste Fehler auf, die ersten 200 Reihen anstelle der gesamten 27k Reihen, verfäscht in diesen Fall aus 2 Gründen die Ergebnisse. Zum einen ist 200%6 =! 0, dementsprechend fehlen Subprocesse des letzten eigentlichen Processes, aber auch betracheten wir weniger als 0,007% der Datenmenge. So können unmöglich auschlagkräftige Ergebnisse erzielt werden, aber für den Zweck dieser kleinen Demonstration ist es genügend.

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/GDCeE2JQW6m2rZz?x=1920&y=579&a=true&file=process_spectrum.png&scalingup=0) 


## Performance Spektrum errechnen

Die ersten komplexeren Berechnungen nimmt uns hierbei die Pandas Libary ab, in dem diese die Eventlogs in zuerst in eine einheitliches und auswertbares Zeitformat umformt. Wir setzen die Startzeiten auf die geringste Dauer, und die Endzeiten auf die höschste der jeweiligen Cases. Anschließend subtrahieren wir Startzeit von der Endzeit, und uns bleibt eine durschnittliche Zeit für die Dauer der jeweiligen Aktivitäten.

Anschließend visualisieren wir unsere Ergebnisse wieder, und geben diese als Bild aus.

In [ ]:
# Calculate case durations
event_log['timestamp'] = pd.to_datetime(event_log['timestamp'])
start_times = event_log.groupby('case_id')['timestamp'].min()
end_times = event_log.groupby('case_id')['timestamp'].max()
case_durations = end_times - start_times

# Plot the performance spectrum
plt.figure()
case_durations.dt.total_seconds().plot(kind='bar', color='lightblue')
plt.xlabel('Case ID')
plt.ylabel('Duration (seconds)')
plt.title('Performance Spectrum')

# Save the performance spectrum plot as an image file
performance_spectrum_file = os.path.join(output_dir, 'performance_spectrum.png')
plt.savefig(performance_spectrum_file)

print(f"Performance spectrum saved as {performance_spectrum_file}")


Auch hierbei ist die Verfälschung deutlich zu erkennen, die einzelnen Subprocesse der Processe wurden unter der gemeinsamen Case ID zusammengefasst. Da die Dauer dieser Processe stark variiert, und der kürzste Teil des Processe nur wenige Sekunden dauert, ist auch mit dem Ergebnisse wenig anzufangen.

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/MgAa2KbbPTRTwsE?x=1920&y=579&a=true&file=performance_spectrum.png&scalingup=0)



Neben den hier verwendeten Libarys sind folgende libarys erwähnenswert, allerdings dauert die Implimentation wesentlich länger und diese Demo ist zeitlich sehr beschränkt.

 - pm4py
 - pmi

## Prom 6.9

Die Prom Plattform ist ein plugin-basierendes Framework speziell für Process Mining. Zentriert auf drei Basiskonzepten. 

Data Objects (1) im Workspace Tab (2)
Plugins (3)
Visualisierung (4)

![alt text](https://promtools.org/wp-content/uploads/2022/12/prom_tabs-1024x346.png)

## Workspace

Ansicht (5) von allen Objekten, den Favorisierten Objekten und Importierten Objekten.

Objekte importierbar über den import button (6) oder via drag&drop.

Im workspace kann außerdem:
(7) favorisiert (Stern), angeschaut (Auge), Plugin gestartet (Play) oder gelöscht (Kreuz),
(8) umbenannt,
(9) exportiert werden.

![alt text](https://promtools.org/wp-content/uploads/2022/12/prom_workspace-1024x576.png)



## Praktische Anwendung
Die Code Demo ist ein wenig kurz an tatsächlichen Code gekommen, aber mir war es wichtiger einen sinnvollen Einblick in die praktische Arbeit mit Process Mining zu geben.

Fangen wir nun mit der Umwandlung der CSV-Datei an. Zualler erst schauen wir uns dazu wie vorab beschrieben den Inhalt an für ein Verständnis der Daten, und klassifizieren die Eingabevariabeln.

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/qCd95sMpQoNxCe4?x=952&y=561&a=true&file=CSV_Klassifizieren.png&scalingup=0)

## Umwandlung in .XES Format

Anschließend übertrage wir die Variabelklassifizierung und alle Spalten und Reihen der CSV-Datei in ein XES Format, dies machen wir mit dem zugehörigen Standard-Plugin von ProM 6.9.

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/X8f8M2TLHABX8zg?x=1920&y=579&a=true&file=CSV_Convert.png&scalingup=0)






## Analyse 

Die XES-Datei können wir problemlos in das Performance Spektrum Plugin einfüttern. 

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/WTBF39MAef4GaGN?x=1920&y=579&a=true&file=PSM_PM_START.png&scalingup=0)

Das Feintuning der Parameter und das setzen eines eigenen Classifier ist optimal, kann aber je nach dem Ziel des Anwenders stark die Ergebnisse berichtigen und verfälschen. Die ersten zwei Ausgabebilder werten die Versicherungsdaten in Quantilen aus:

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/dH9959osKBkyGXk?x=1920&y=579&a=true&file=PSM_Quartile_1.png&scalingup=0)
![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/PLenw6S3zF3CxtE?x=1920&y=579&a=true&file=PSM_UANTILE_2.png&scalingup=0)

hier die mediale Auswertung:

![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/bg9y3t9gdikANSq?x=1920&y=579&a=true&file=PSM_Medianbased1.png&scalingup=0)
![alt text](https://nextcloud.th-deg.de/apps/files_sharing/publicpreview/cLfPWsQ4mLgGyYK?x=1920&y=579&a=true&file=PSM_MEDIAN_3.png&scalingup=0)

## Auswertung

Zur Auswertung nehme ich den medialen Ansatz her, da er hierbei mehr Sinn ergibt. Man kann relativ deutlich erknnen bei der 2. Abbildung, dass die ersten Prozesse die erste Benachrichtigung über den Verlust ist, und die letzte das Abschließen durch abgeschlossene Bezahlung oder durch Entscheidung keine Zahlung zu senden ist. Ebenfalls zu sehen ist, dass die meisten sehr (Dunkelblau) und langsamen (Orange) sich innerhalb sehr speziellen Prozessen befinden, während die häufigsten Prozesse (First Notification) schon sehr gut optimiert sind und komplett normal im Durschnitt ablaufen.
In der ersten Abildung ist ebenfalls zu sehen, wie verschiedene Prozesse sich überschneiden. Die Linien stellen jeweils einen Prozess von Beginn bis Ende da. Dies ist bei den oft vorkommenden Prozessen leider sehr unübersichtlich, während es bei den weniger Häufigen Prozessen zu wesentlich besser sichtbaren Resultaten führt.